# Worklog

## 2019-04-03 to 2019-04-05

Rereading lecture slides from Language Technology [EDAN20](http://cs.lth.se/edan20/lectures/).

Looking articles/websites/packages:

- [Semantic annotation in biomedicine: the current landscape](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5610427/)
- [A Survey of Bootstrapping Techniques in Natural Language Processing](https://www.eecis.udel.edu/~vijay/fall13/snlp/lit-survey/Bootstrapping.pdf)
- [POS-tagger and NER for bio-medical abstracts](http://www.nactem.ac.uk/tsujii/GENIA/tagger/)
    - [Developing a Robust Part-of-Speech Tagger for Biomedical Text]()


## 2019-04-10

Vilhelm is sick, so Olof and Vilhelm talk over VOIP and are installing software so they can start working.

Discussion how to approaching solving detecting relationships - _A inhibits B_. Come up with this plan:

1. Use Genia parser to generate Part-Of-Speech tags for all words in the abstracts.
    - GENIA parser was created for this task specifically: it's based on a feature set tailored to working on regular text and biomedical abstracts. Was then trained on a regular corpus as well as biomedical corpora.
1. Use the Stanford Core NLP package to generate dependency graphs
1. Use dependency graph to find the relationships.
    - Not sure how to do this yet, but we believe this should be possible. We think the head of the relevant sentences will be _"inhibits"_ or similar, and then the inhibitor and inhibitee will be directly related to it in the depency graph. 

### Potential problems

-  Generating POS's and a dependency graph for each abstract in the database might take an extremely long time. We might be able to get around this by filtering sentences
- Pierre said two different NLP packages might not always work well together. Hopefully this is just because of a differing POS scheme so we can add a POS-scheme translator between the POS tagger and the Dependency parser.
    - Apparently the Stanford CoreNLP package [can take annotated data](https://stanfordnlp.github.io/CoreNLP/cmdline.html#inputting-serialized-files), so this approach might still be viable.
    - We should maybe run a POS annotation test and compare if the tags are similar?


### Articles found

- [NLP-progress: Relationship Extraction](http://nlpprogress.com/english/relationship_extraction.html)
    - Might be useful for detecting relationships from dependency graphs?

## 2019-04-11

### Vilhelm
Created script to extract abstracts from the Pubmed XML files. The PMID of each article is attached in the generated output to enable easily finding the original article. Using NLTK to split abstracts into one sentence per line as this is necessary for GENIA Parser.

## 2019-04-12
### Vilhelm
Stanford Core NLP recommended to run their toolkit in batches instead of individual sentences, as it has some loading time.

Olof only seemed to be able to run the Stanford Core NLP in smaller batches, so I cleaned up the abstract extractor and added batch functionality.

Got a script from Olof regarding running and using Stanford Core NLP and tweaked it to take random batches of abstracts and run POS-tagging and Dependency parsing on them.

- Dependency-graph head detection seems not to be working right. From a biomedical abstract with 6 sentences (PMID: 5859018), the dependency parser registered 6 heads, all of them in the first sentence.
- POS-tagging with Stanford Core NLP seems to be working alright. From the same abstract we retrieved the following tags. I don't know much about medical terms, but I guess this looks right?

```
The	DT
incorporation	NN
of	IN
tritiated	JJ
thymidine	NN
into	IN
the	DT
DNA	NN
of	IN
erythroblasts	NNS
,	,
primitive	JJ
ependymal	JJ
cells	NNS
,	,
and	CC
mesenchymal	JJ
cells	NNS
of	IN
.....```

- The XML parser retrieved som weird stuff from _pubmed19n0195.xml_, one abstract had an author list and citations in it, which messed up the sentence splitter from NLTK, and thus messed up stuff for Stanford Core NLP.